In [26]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import itertools

import re
import nltk
import pandas as pd
from nltk.stem import WordNetLemmatizer 

from scipy.sparse import csr_matrix

from sklearn.decomposition import LatentDirichletAllocation as LDA

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
tweets_frame = pd.read_csv('../get-tweets-geolocation/src/training_tweets_for_EDA.csv')

In [37]:
class Tweets_LDA():
    
    '''
    Class gets as input raw-data from twitter in csv format.
    There realized methods for data preprocessing and perfoming LDA on the preprocessed data.
    As output - words that are indicators of particular topic for tweets.
    '''
    
    def __init__(self,):
        self.path_to_file = ''
        self.tweets_frame = pd.DataFrame()
        
    def load_data(self,path_to_file):
        self.path_to_file = path_to_file
        self.tweets_frame = pd.read_csv(self.path_to_file)
        self.tweets_frame = self.tweets_frame[self.tweets_frame['created_at'].map(lambda x: x[:10])=='Sat Jun 01']
        
        
    def data_preprocessing(self):
        WPT = nltk.WordPunctTokenizer()
        lemmanizer = WordNetLemmatizer()
        stop_word_list = nltk.corpus.stopwords.words('english')
        
        self.tweets_frame['tweets_processed'] =\
                self.tweets_frame['tweet'].map(lambda x: [lemmanizer.lemmatize(y) for y in re.sub("[\d+0-9.…#!'\"_?,;/:()’%*🤯“”&🧨$🧨🧡]", "", x.lower()).split() 
                                         if y not in stop_word_list 
                                         and y not in ['・・・','','-']
                                         and not y.startswith('http')
                                         and not y.startswith('@')])
        
        self.tweets_frame['tweets_processed'] =\
                self.tweets_frame['tweets_processed'].map(lambda x: [y for y in x if len(y)>2])
        
        self.tweets_frame = self.tweets_frame[self.tweets_frame['tweets_processed'].map(len)>0]
        
    def LDA(self,n_topics):
        
        def create_co_occurences_matrix(allowed_words, documents):
            word_to_id = dict(zip(allowed_words, range(len(allowed_words))))
            documents_as_ids = [np.sort([word_to_id[w] for w in doc if w in word_to_id]).astype('uint32') for doc in documents]
            row_ind, col_ind = zip(*itertools.chain(*[[(i, w) for w in doc] for i, doc in enumerate(documents_as_ids)]))
            data = np.ones(len(row_ind), dtype='uint32')  # use unsigned int for better memory utilization
            max_word_id = max(itertools.chain(*documents_as_ids)) + 1
            docs_words_matrix = csr_matrix((data, (row_ind, col_ind)), shape=(len(documents_as_ids), max_word_id))  # efficient arithmetic operations with CSR * CSR
            words_cooc_matrix = docs_words_matrix.T * docs_words_matrix  # multiplying docs_words_matrix with its transpose matrix would generate the co-occurences matrix
            words_cooc_matrix.setdiag(0)
            return words_cooc_matrix, word_to_id 
        
        def print_topics(model, count_vectorizer, n_top_words,words):
            #words = cross_tab.columns
            for topic_idx, topic in enumerate(model.components_):
                print("\nTopic #%d:" % topic_idx)
                print(" ".join([words[i]
                                for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
        merged = list(itertools.chain.from_iterable(self.tweets_frame['tweets_processed'].values))
        merged_cnts = np.unique(merged,return_counts=True)
        words = merged_cnts[0][merged_cnts[1]>2]
        a, b = create_co_occurences_matrix(words,self.tweets_frame['tweets_processed'].values)
        lda_ = LDA(n_components=n_topics)
        lda_.fit(a)
        print_topics(lda_,' ',60,words)
        
    

In [38]:
a = Tweets_LDA()

In [39]:
a.load_data('../get-tweets-geolocation/src/training_tweets_for_EDA.csv')

In [40]:
a.data_preprocessing()

In [41]:
a.LDA(5)


Topic #0:
traffic back yankee lane blocked stop min near driver delay accident ave game bike red may june ball win queen sox manhattan final stadium right pkwy liverpool two community stopped america champion blvd left approaching league mets bill team lead run uclfinal park show get tonight one amp hbo state sayin tit first claiming vehicle nyc baseball today board unofficial

Topic #1:
incident cleared construction avenue street updated station exit direction boulevard qbus road archer new sutphin line queen parkway expressway east closure plaza jersey south level york bridge ave cortelyou north rte event west toll georgewashingtonbridge park island special central bound hill inysthruway ocean washington side belt ramp bbus downtown route flushing bmbus county center interchange bronx mbus richmond penn sideupper

Topic #2:
new york nyc happy pride today amp month tonight day photo city brooklyn posted jersey park june saturday time chance love sat t-storm jun forecast birthday nigh

In [42]:
a.LDA(10)


Topic #0:
day night time amp today last get one new year great first game like good yankee tonight got nyc see come love back summer going home morning let two week best saturday month happy red next weekend take york way every june show make music friend tomorrow win ive play run work team cant girl still watch start season high

Topic #1:
like dont get one people know time love really cant shit amp make see got day good need lol think never would thats want say life new even thing today still year much ive going right look yall back youre always way man let feel take ever come work friend best guy first someone nigga great happy every lmao better

Topic #2:
case freeship liverpool champion final movie amp uclfinal league one state clear win watching team fan time dvd white get watch today record like best sleeve would match year see make whentheyseeus american generic john color day black let school watched love jewel come many plastic game double full medium paper boy episode well 